In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0895233154296875
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 10.906234965592017
200000 18.1040297355032
300000 18.978194806049192
400000 20.207501631113217
500000 21.580106658799217
600000 20.250384792869415
700000 21.17622852664683
800000 19.208539805747584
900000 19.07677481030935
1000000 19.1631549233192
1100000 20.27669613694618
1200000 17.24310949883933
1300000 22.579713114856375
1400000 3.866049236183496
1500000 23.490420075798443
1600000 24.580035160343677
1700000 24.915097089162142
1800000 19.380775763689748
1900000 22.252544465583448
2000000 16.64598189445361
2100000 24.597662810924575
2200000 19.4691861464769
2300000 23.702829170674217
2400000 22.226691868950486
2500000 28.534480872143806
2600000 32.49436491493379
2700000 21.851092199197733
2800000 29.693652123206913
2900000 55.12430068416477
3000000 52.3472

29700000 38.41269421859305
29800000 13.428735221023587
29900000 35.76203111336489
30000000 34.75968973059193
30100000 37.69744240221876
30200000 33.74588351883099
30300000 36.426565044172385
30400000 34.59573624197809
30500000 35.75871744934797
30600000 32.64266109171829
30700000 36.21035783033863
30800000 31.122737354445555
30900000 16.86411104944292
31000000 27.544491438004542
31100000 33.72425688905191
31200000 36.07710168346096
31300000 37.74117640531775
31400000 35.58411154450122
31500000 34.37826658630955
31600000 34.71203989745767
31700000 35.86194399117729
31800000 36.46233679117453
31900000 35.715950593514236
32000000 25.36728801068147
32100000 23.94561870037014
32200000 38.60132510673225
32300000 36.426152787735795
32400000 37.77031160286656
32500000 15.061539741938002
32600000 33.80216306533071
32700000 36.18279427066906
32800000 36.87740852669522
32900000 35.6912239700566
33000000 37.231359299616685
33100000 34.061208171972616
33200000 37.53198071469596
33300000 34.71617720

59700000 33.39825713222593
59800000 32.71349474109528
59900000 31.99216035807698
60000000 31.593878642131997
60100000 31.924092093711387
60200000 30.908787792392904
60300000 30.76613188738917
60400000 25.639294374123413
60500000 12.560452840050608
60600000 31.208827798577833
60700000 30.480771064573307
60800000 31.829276026238357
60900000 30.66388859334492
61000000 33.549094566579804
61100000 31.518476176827853
61200000 33.80155207434302
61300000 30.92044632136114
61400000 14.208110662463419
61500000 33.26930880655528
61600000 34.877441143970195
61700000 33.110996762373524
61800000 34.614685861277046
61900000 35.14352826102914
62000000 33.66211394912461
62100000 34.75056596619816
62200000 29.11147326406804
62300000 16.557138263862125
62400000 32.66482503797479
62500000 32.87759084605838
62600000 32.57069946112931
62700000 34.21907815277697
62800000 31.247998741007095
62900000 30.754531317102654
63000000 30.574553093701244
63100000 21.853350458057406
63200000 17.387875225143155
63300000

89500000 27.27084046072497
89600000 27.51790765582576
89700000 27.657680417846258
89800000 27.577760918706552
89900000 27.123548291521203
90000000 9.136791165012475
90100000 25.743743849996147
90200000 27.360748837593356
90300000 27.096658414962278
90400000 26.50667462116783
90500000 27.367240929256358
90600000 27.18546904723374
90700000 11.268812569166792
90800000 27.37390362804028
90900000 29.447462532064566
91000000 29.99497263321595
91100000 29.90926397035369
91200000 29.865493340490403
91300000 31.79662850754526
91400000 31.004530537238022
91500000 28.989495288059274
91600000 29.061657956904252
91700000 29.168041954293113
91800000 28.135172076573056
91900000 28.02573664365429
92000000 13.181561355495324
92100000 21.343629929489364
92200000 26.305373307626947
92300000 25.822285346161976
92400000 25.33102631814055
92500000 25.648447313254234
92600000 27.167051683707456
92700000 4.6479921734136305
92800000 27.824818427218595
92900000 25.772866414552244
93000000 25.674128394969
931000

118600000 25.595935980958934
118700000 25.284584439330576
118800000 23.08369434827115
118900000 23.474211320515913
119000000 19.757189777121415
119100000 10.883338482839513
119200000 22.518044099719972
119300000 22.587118302452556
119400000 22.99436007671185
119500000 16.773890152558362
119600000 13.811916739390218
119700000 23.16427767661916
119800000 22.98729578335051
119900000 22.770135590673128
120000000 14.553336482838326
120100000 17.415804592614545
120200000 22.99805458385643
120300000 23.42634795324001
120400000 24.810273828208665
120500000 24.067424365104753
120600000 25.363670455913827
120700000 25.469408777498863
120800000 25.68959953361454
120900000 25.114482553419197
121000000 25.526277721108883
121100000 25.405807733609095
121200000 25.060735939730545
121300000 25.390967552216974
121400000 25.180159462824815
121500000 22.94113176741075
121600000 22.871418497697753
121700000 21.11539292974134
121800000 9.947733410655504
121900000 22.36211416546056
122000000 23.644720687642

...

233100000 14.339166506257648
233200000 14.01089025502495
233300000 14.327097546555896
233400000 14.796570953073191
233500000 15.018992421861023
233600000 14.710032909621578
233700000 14.584661698687855
233800000 14.446527931555673
233900000 14.359962446363706
234000000 14.11797963907851
234100000 13.66463975030167
234200000 12.884955026373365
234300000 13.257007027214863
234400000 12.242803759462479
234500000 13.417988826920404
234600000 12.589170374580759
234700000 4.852136747241983
234800000 12.70980854561902
234900000 12.743679372765527
235000000 7.183858539574812
235100000 9.171288351057827
235200000 12.504943696158207
235300000 12.556853350883678
235400000 4.426945808919583
235500000 13.511598521201075
235600000 12.31731790261436
235700000 13.47822122835295
235800000 12.82803359281105
235900000 13.511399966315429
236000000 12.785402562150301
236100000 14.611027665999888
236200000 13.751271644866147
236300000 14.151268618284282
236400000 14.017309430975763
236500000 14.255201208487

261700000 9.831935993029404
261800000 10.883223675636893
261900000 1.7521615568713975
262000000 10.583069859905853
262100000 10.170705569132565
262200000 10.347594170836247
262300000 9.861694076607607
262400000 6.15106562784387
262500000 8.134657116898943
262600000 9.971558734574131
262700000 10.85755513615821
262800000 10.540128199261856
262900000 11.765922252969654
263000000 10.368640743472403
263100000 11.38806231806376
263200000 10.566592084009024
263300000 12.217061197072386
263400000 10.448473290823404
263500000 11.789422029188298
263600000 10.385358389670754
263700000 11.631382028204888
263800000 10.134976995693043
263900000 10.656609794076447
264000000 4.853503153157615
264100000 9.111867030277857
264200000 9.922893512206214
264300000 10.192182635687587
264400000 9.724578687015864
264500000 10.415763117321927
264600000 3.483065061388604
264700000 10.32962038888477
264800000 10.00136050333586
264900000 9.673215723133739
265000000 9.874226724349526
265100000 6.223586087843541
265

290900000 8.115474483360044
291000000 7.259657724487569
291100000 7.828274946327066
291200000 7.4547645963666795
291300000 7.852596752480535
291400000 4.299419462414613
291500000 6.082989974267615
291600000 7.297229518230796
291700000 8.096548629660793
291800000 7.05634963381939
291900000 8.175374831747922
292000000 7.485872032799503
292100000 8.513977788150651
292200000 2.473691252463344
292300000 8.763412012062936
292400000 8.117174207594044
292500000 8.670537692771122
292600000 7.860181783040123
292700000 8.905635578931903
292800000 8.05026002366246
292900000 8.654532980317473
293000000 8.141409824632143
293100000 8.172108711306963
293200000 8.103889256015066
293300000 5.693397817493153
293400000 5.7024832655013595
293500000 7.710671560955485
293600000 7.555071070999987
293700000 7.019699491430197
293800000 7.272259026835057
293900000 7.329458474684277
294000000 7.370367908733709
294100000 3.1566162612476685
294200000 6.569669786210352
294300000 7.266697264693269
294400000 7.6932613

320100000 5.781455329959289
320200000 5.393687710719453
320300000 5.849305984738905
320400000 4.987709051193476
320500000 5.718368352702265
320600000 4.95291470377362
320700000 5.333935666389749
320800000 4.827204828542203
320900000 4.6143440571394505
321000000 1.9611275350707047
321100000 5.102312137447115
321200000 4.91385467658636
321300000 5.1161816641469215
321400000 5.36242465619364
321500000 5.658659978017731
321600000 5.2801681932936875
321700000 5.51009907033766
321800000 5.552882838540606
321900000 3.695468175537764
322000000 3.9258499416442443
322100000 5.474602612277574
322200000 5.401914870393169
322300000 5.304010687651637
322400000 5.477759277666202
322500000 5.188877453244667
322600000 4.840340885787509
322700000 2.3500507551215946
322800000 5.588272374963588
322900000 5.204707527887128
323000000 5.323454115056402
323100000 4.793054400571429
323200000 5.268596416715229
323300000 4.733281837035614
323400000 4.94716276934006
323500000 4.657299389152276
323600000 3.7038007

349200000 2.5107807962338895
349300000 1.7057736677013955
349400000 2.036978171726239
349500000 2.7210421345306677
349600000 2.500932980507454
349700000 2.6626113098623434
349800000 2.57787436900353
349900000 2.5275179432280517
350000000 2.513612804868335
350100000 2.4847072993468426
350200000 2.5138568639914785
350300000 2.5568794643937114
350400000 2.4170242075953285
350500000 0.9298326540339403
350600000 2.146873327131736
350700000 2.497817386281868
350800000 2.5186794769842176
350900000 2.3407622572845876
351000000 2.530049314240149
351100000 2.3606077248459827
351200000 2.583423558148279
351300000 2.3963294328487805
351400000 2.550307217162838
351500000 2.3998774238036455
351600000 2.7363984104887837
351700000 0.5841009572752516
351800000 2.746034248801017
351900000 2.3782869763157737
352000000 0.9047226799114784
352100000 2.414032666041603
352200000 2.527084794488443
352300000 2.2831465682813796
352400000 2.4582165794265984
352500000 2.3878505762380566
352600000 2.315326395868368